In [14]:
import numpy as np
import pandas as pd
from trie import Trie
import re

In [3]:
# 车型词库使用 172.26.171.57 /home/songyunsheng/dict/t_veco_brand_series.csv
veco_brand_series = pd.read_excel('t_veco_brand_series.xlsx', na_values='NA')
veco_brand_series = veco_brand_series[['BRAND', 'CAR_SERIES', 'KEY_WORD']].dropna().drop_duplicates()

In [15]:
veco_trie = Trie(reverse=False)
for i, row in veco_brand_series.iterrows():
    try:
        veco_trie.insert(row['KEY_WORD'].strip())
    except:
        print(row)

BRAND             长安
CAR_SERIES    长安CS75
KEY_WORD          75
Name: 7951, dtype: object


In [80]:
dff = pd.read_excel('ann_sent_range_pred.xlsx')
dff_vehicle = dff[['sentence_list_pred','sentence_list']].fillna('[]')
dff_vehicle.head()

,sentence_list_pred,sentence_list
0,轩逸(10:12),「天天3.15」日产轩逸新车刚买俩月频繁提示故障！
1,[],辽源车主要求换车二三里资讯松原1598930477
2,[],买车本是为节省时间并相对提高生活质量，可新车刚刚到手却三番五次出现故障，还能淡定吗？
3,轩逸(33:35),"2020年8月30日，辽源市民隋先生向二三里资讯求助：""我买的日产轩逸汽车刚开两个月，出现了..."
4,[],消费者：


In [81]:
import re
def Correct_vehicle(vehicles,sentences,Dict=CAR_SERIES,location=False):
    MAX_LEN = 0
    for item in Dict:
        if len(item) > MAX_LEN:
            MAX_LEN = len(item)
    correct_vehicle = []
    for vs,s in zip(vehicles.values,sentences.values):
        tmp = []
        for v in vs.split('|'):
            if v != '[]':
                v_name = re.search(r"(.*?)\(",v).group(1)
                start = int(re.search(r"\d+:\d+",v).group().split(':')[0])
            else:
                continue
            if  v_name not in Dict:
#                 start = re.search(v, s).span()[0]
                window = min(len(s), start + MAX_LEN)
                for i in range(window, start, -1):
                    if s[start:i] in Dict:
                        if not location:
                            tmp.append(s[start:i])
                            print(s[start:i])
                            break
                        else:
                            tmp.append(s[start:i]+'({}:{})'.format(start,i))
                            print(s[start:i]+'({}:{})'.format(start,i))
                            break
            else:
                if not location:
                    tmp.append(v_name)
                else:
                    tmp.append(v)

        correct_vehicle.append('|'.join(list(set(tmp))))
    return correct_vehicle

In [82]:
correct_vehicle = Correct_vehicle(dff_vehicle['sentence_list_pred'],dff_vehicle['sentence_list'],location=True)
correct_vehicle = [v if v != '' else '[]' for v in correct_vehicle ]

艾瑞泽5(2:6)
凯美瑞(4:7)
锐界(49:51)
纳瓦拉(14:17)
城市(33:35)
阿特兹(7:10)
马自达CX-4(4:11)
标致408(4:9)
领动(26:28)
锐界(49:51)
纳瓦拉(14:17)
城市(33:35)
普桑(26:28)
标致408(127:132)
领动(24:26)
标致408(127:132)
领动(31:33)


In [83]:
correct_vehicle_ = pd.DataFrame(correct_vehicle, columns=['修正预测车型']) 
dff = pd.concat([dff, correct_vehicle_], axis=1)
dff.head()

,article_id,paragraph_id,sentence_id,有无,sentence_list,sentence_list_pred,修正预测车型
0,0,0,0,有,「天天3.15」日产轩逸新车刚买俩月频繁提示故障！,轩逸(10:12),轩逸(10:12)
1,0,0,1,无,辽源车主要求换车二三里资讯松原1598930477,NaN,[]
2,0,1,0,无,买车本是为节省时间并相对提高生活质量，可新车刚刚到手却三番五次出现故障，还能淡定吗？,NaN,[]
3,0,2,0,有,"2020年8月30日，辽源市民隋先生向二三里资讯求助：""我买的日产轩逸汽车刚开两个月，出现了...",轩逸(33:35),轩逸(33:35)
4,0,3,0,无,消费者：,NaN,[]


In [85]:
dff[dff['修正预测车型'] != '[]'].tail(20)

,article_id,paragraph_id,sentence_id,有无,sentence_list,sentence_list_pred,修正预测车型
749,31,7,0,有,其中一位2019款标致408车主称，车辆在冷启动后发动机哒哒响声音很大，几分钟后消失，消失后...,标致408(9:14)|标(127:128),标致408(127:132)|标致408(9:14)
750,31,8,0,有,NO.2 一汽-大众迈腾,迈腾(10:12),迈腾(10:12)
751,31,9,0,有,一汽-大众是投诉榜上的重灾区，旗下三款车型进入前十，包括速腾、迈腾以及探岳三款车型，其中速腾...,速腾(44:46)|探岳(35:37)|速腾(28:30)|迈腾(31:33),迈腾(31:33)|速腾(44:46)|速腾(28:30)|探岳(35:37)
752,31,9,1,有,速腾共消费者提供了1.2T和1.4T两种选择，均匹配双离合变速器，这种变速器虽说传动效率高，...,速腾(0:2),速腾(0:2)
753,31,10,0,有,而双离合变速器又分为干式和湿式两种，前者产生顿挫的可能性更大，而据说速腾据说采用的就是干式。,速腾(34:36),速腾(34:36)
754,31,10,1,有,一位2019款速腾车主投诉称，车辆变速器不仅出现异响，还存在换挡顿挫的问题，其别4s店告知属...,速腾(7:9),速腾(7:9)
756,31,12,0,有,NO.3 东风日产奇骏,奇骏(9:11),奇骏(9:11)
757,31,13,0,有,东风日产奇骏投诉量排第十位，但由于大众探岳和现代领动投诉量相同，于是按并列第9名计算，奇骏则...,探岳(19:21)|奇骏(43:45)|领(24:25)|奇骏(4:6),领动(24:26)|奇骏(43:45)|奇骏(4:6)|探岳(19:21)
758,31,14,0,有,比如一位2019款车主投诉称，其车辆在车速20~30km/h时，加油有顿挫感，其发动机舱有哒...,奇骏(52:54),奇骏(52:54)
767,32,4,0,有,标致408(参数|图片)曾经是国内轿车市场上的热销车型，2016年时月销一度破万，而在今年7...,标致408(0:5)|标致408(48:53),标致408(0:5)|标致408(48:53)


In [110]:
dff.to_csv('correct_vehicle_withloc.csv',encoding='utf-8-sig',index=False)

## 期望的解决方案：

In [77]:
import re

def correct_vehicle(vehicles, sentences, trie, location=True):
    """
    根据车型词典修正车型实体识别模型只能识别出头部的车型。
        -vehicles：预设的车型信息，以'|'分隔不同车型, location=True时车型信息要包含索引
        -sentences：目标文本
        -trie:车型字典（新增方法search_starts：广度优先遍历trie树，获取以某个字符串开头词的列表）
        -location：是否包含索引信息，True->包含，False->不包含
    """
    corrected_vehicle = []
    correct_jobs = []
    for v in vehicles.split('|'):
        if not v:
            continue
        if location:
            v_name, v_start_id, v_end_id = re.search('(.*?)\((\d*?):(\d*?)\)', v).groups()
            v_start_id, v_end_id = int(v_start_id), int(v_end_id)
            correct_jobs.append([v_name, v_start_id])
        else:
            v_name = v
            for r in re.finditer(v, sentences):
                v_start_id, v_end_id = r.span()
                correct_jobs.append([v_name, v_start_id])
    correct_jobs = sorted(correct_jobs, key=lambda x: len(x[0]), reverse=True)
    for v_name, v_start_id in correct_jobs:
        candidates = trie.search_starts(v_name)
        for v in candidates:
            if sentences[v_start_id:].startswith(v):
                if location:
                    corrected_vehicle.append('{}({}:{})'.format(v, v_start_id, v_start_id+len(v)))
                else:
                    corrected_vehicle.append(v)
                break
    return '|'.join(corrected_vehicle)

dff = pd.read_excel('ann_sent_range_pred.xlsx')
dff_vehicle = dff[['sentence_list_pred','sentence_list']].fillna('')
dff_vehicle['corrected_vehicle'] = dff_vehicle.apply(
    lambda row: correct_vehicle(row['sentence_list_pred'], row['sentence_list'], veco_trie, location=True),
    axis=1)

dff_vehicle[dff_vehicle['corrected_vehicle'] != ''].tail(20)